In [1]:
from sqlalchemy import create_engine, Column, Date, String, Integer, Float, exc, and_, or_
from sqlalchemy.orm import Session, declarative_base

import requests

from selectorlib import Extractor
import requests
import json
from time import sleep
import random
import re


In [2]:
engine = create_engine("sqlite+pysqlite:///books.sqlite")
Base = declarative_base()
Base.metadata.create_all(engine)
session = Session(engine)

In [3]:

N = 3


def similarity(set1:set, set2:set, thresh = 0.8):
    #print(len(set1.intersection(set2)) / len(set1.union(set2)))
    try:
        return (len(set1.intersection(set2)) / len(set1.union(set2))) > thresh
    except:
        return 0

def similarity_complete(text1:String, text2:String, thresh = 0.8):

    set1 = set([text1.lower()[i:i+N] for i in range(len(text1)-N+1)])
    set2 = set([text2.lower()[i:i+N] for i in range(len(text2)-N+1)])
    #print(text1)
    #print(text2)
    return similarity(set1, set2, thresh)

In [4]:
class Amazon(Base):
    __tablename__ = 'Amazon'

    isbn = Column(String, primary_key=True)
    title = Column(String)
    subtitle = Column(String)
    author = Column(String)
    description = Column(String)
    book_type = Column(String)
    price = Column(Float)
    rating = Column(Float)
    review_count = Column(Integer)
    page_count = Column(Integer)
    language = Column(String)
    publisher = Column(String)
    published_date = Column(String)
    url = Column(String)


class Goodread(Base):
    __tablename__ = 'Goodreads'

    isbn = Column(String, primary_key=True)
    rating = Column(Float)
    rating_count = Column(Integer)
    review_count = Column(Integer)


class GoogleBook(Base):
    __tablename__ = 'GoogleBooks'

    isbn = Column(String, primary_key=True)
    title = Column(String)
    subtitle = Column(String)
    authors = Column(String)
    description = Column(String)
    categories = Column(String)
    average_rating = Column(Float)
    rating_count = Column(Integer)
    maturity_rating = Column(String)
    language = Column(Integer)
    page_count = Column(Integer)
    publisher = Column(String)
    published_date = Column(String)


class NYT(Base):
    __tablename__ = 'NYT'

    isbn = Column(String, primary_key=True)
    title = Column(String, nullable=False)
    author = Column(String, nullable=False)
    score = Column(Integer, nullable=False)
    publisher = Column(String, nullable=False)
    book_image = Column(String)
    amazon_product_url = Column(String)


class MergedBooksUpdated(Base):
    __tablename__ = 'merged_books_updated'

    isbn = Column(String, primary_key=True)
    title = Column(String, nullable=False)
    author = Column(String, nullable=False)
    publisher = Column(String)
    published_date = Column(String)
    description = Column(String)
    maturity_rating = Column(Integer)
    book_type = Column(String)
    price = Column(Float)
    page_count = Column(Integer)
    score_nyt = Column(Integer)
    goodreads_rating = Column(Float)
    goodreads_count = Column(Integer)
    amazon_rating = Column(Float)
    amazon_count = Column(Integer)
    google_rating = Column(Float)
    google_count = Column(Integer)
    book_image = Column(String)
    amazon_link = Column(String)

In [5]:
for book in session.query(NYT).all():

    isbn = book.isbn

    nyt = session.query(NYT).filter(NYT.isbn == isbn).first()
    amazon = session.query(Amazon).filter(Amazon.isbn == isbn).first()
    goodread = session.query(Goodread).filter(Goodread.isbn == isbn).first()
    googlebook = session.query(GoogleBook).filter(GoogleBook.isbn == isbn).first()



    isbn = nyt.isbn

    title = nyt.title
    if amazon.title != None and  not similarity_complete(nyt.title, amazon.title, 0.5):
        title += f"\nAlternate: {amazon.title}"
    if googlebook.title != None and not similarity_complete(nyt.title, googlebook.title, 0.5):
        title += f"\nAlternate: {googlebook.title}"


    author = nyt.author
    if amazon.author != None and not similarity_complete(nyt.author, amazon.author, 0.5):
        author += f"\nAlternate: {amazon.author}"
    if googlebook.authors != None and not similarity_complete(nyt.author, googlebook.authors, 0.5):
        author += f"\nAlternate: {googlebook.authors}"

    publisher = nyt.publisher
    if amazon.publisher != None and not similarity_complete(nyt.publisher, amazon.publisher, 0.5):
        publisher += f"\nAlternate: {amazon.publisher}"
    if googlebook.publisher != None and not similarity_complete(nyt.publisher, googlebook.publisher, 0.5):
        publisher += f"\nAlternate: {googlebook.publisher}"


    
   
    published_date = amazon.published_date
    description = googlebook.description
    maturity_rating = googlebook.maturity_rating
    book_type = amazon.book_type
    price = amazon.price

    page_count = amazon.page_count
    if page_count == None:
        page_count = googlebook.page_count

    score_nyt = nyt.score

    goodreads_rating = goodread.rating
    goodreads_count = goodread.rating_count

    amazon_rating = amazon.rating
    amazon_count = amazon.review_count

    google_rating = googlebook.average_rating
    google_count = googlebook.rating_count

    book_image = nyt.book_image
    amazon_link = nyt.amazon_product_url

    book = MergedBooksUpdated(
    isbn = isbn,
    title = title,
    author = author,
    publisher = publisher,
    published_date = published_date,
    description = description,
    maturity_rating = maturity_rating,
    book_type = book_type,
    price = price,
    page_count = page_count,
    score_nyt = score_nyt,
    goodreads_rating = goodreads_rating,
    goodreads_count = goodreads_count,
    amazon_rating = amazon_rating,
    amazon_count =amazon_count,
    google_rating = google_rating,
    google_count = google_count,
    book_image = book_image,
    amazon_link = amazon_link
    )

    session.add(book)
    session.commit()

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: merged_books_updated.isbn
[SQL: INSERT INTO merged_books_updated (isbn, title, author, publisher, published_date, description, maturity_rating, book_type, price, page_count, score_nyt, goodreads_rating, goodreads_count, amazon_rating, amazon_count, google_rating, google_count, book_image, amazon_link) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: ('9781984818515', 'BETTER OFF DEAD\nAlternate: Better Off Dead A Jack Reacher Novel', 'Lee Child and Andrew Child\nAlternate: Lee Child', 'Delacorte', 'October 26, 2021', "#1 NEW YORK TIMES BESTSELLER • Jack Reacher is back in a brand-new page-turning thriller from acclaimed #1 bestselling authors Lee Child and Andrew C ... (1053 characters truncated) ... doncker down will be the riskiest job of Reacher's life. Failure is not an option, because in this kind of game, the loser is always better off dead.", 'NOT_MATURE', 'Hardcover', 17.03, '336 pa

In [14]:
session.rollback()

In [7]:
class MergedBooksAggregatedUpdated(Base):
    __tablename__ = 'merged_books_aggregated_updated'

    isbn = Column(String, primary_key=True)
    title = Column(String, nullable=False)
    author = Column(String, nullable=False)
    publisher = Column(String)
    published_date = Column(String)
    description = Column(String)
    maturity_rating = Column(Integer)
    book_type = Column(String)
    price = Column(Float)
    page_count = Column(Integer)
    score_nyt = Column(Integer)
    goodreads_rating = Column(Float)
    goodreads_count = Column(Integer)
    amazon_rating = Column(Float)
    amazon_count = Column(Integer)
    google_rating = Column(Float)
    google_count = Column(Integer)
    book_image = Column(String)
    amazon_link = Column(String)
    id_flag = Column(Integer)

In [10]:
#sort on title so that we can later use blocking
books = session.query(MergedBooksUpdated).order_by(MergedBooksUpdated.title).all()

In [15]:
finished_books = set()


#blocking where we use a window size of 10 on neighboorhoods which are sorted by title
for count, book1 in enumerate(books):
    print(book1.title)
    
    if book1 in finished_books:
        continue
    relevant_books = [book1]
    print(f"{book.title}, {book.author}")
    grams_title = set([book1.title[i:i+N] for i in range(len(book1.title)-N+1)])
    grams_author = set([book1.author[i:i+N] for i in range(len(book1.author)-N+1)])
    
    for book2 in books[count:count+10]:
        print(book2.title)
        grams_title2 = set([book2.title[i:i+N] for i in range(len(book2.title)-N+1)])
        grams_author2 = set([book2.author[i:i+N] for i in range(len(book2.author)-N+1)])

        if similarity(grams_title, grams_title2) & similarity(grams_author, grams_author2, 0.5):
            books.append(book2)
            finished_books.add(book2)
            
        
    for book in relevant_books:
        session.add(
        MergedBooksAggregatedUpdated(
        isbn = book.isbn,
        title = book.title,
        author = book.author,
        publisher = book.publisher,
        published_date = book.published_date,
        description = book.description,
        maturity_rating = book.maturity_rating,
        book_type = book.book_type,
        price = book.price,
        page_count = book.page_count,
        score_nyt = book.score_nyt,
        goodreads_rating = book.goodreads_rating,
        goodreads_count = book.goodreads_count,
        amazon_rating = book.amazon_rating,
        amazon_count = book.amazon_count,
        google_rating = book.google_rating,
        google_count = book.google_count,
        book_image = book.book_image,
        amazon_link = book.amazon_link,
        id_flag = count
        )
        )
    
    session.commit()
        

'TIL DEATH DO US PART
'TIL DEATH DO US PART, Amanda Quick
'TIL DEATH DO US PART
10TH ANNIVERSARY
Alternate: 10th Anniversary (Women's Murder Club, 10)
11/22/63
Alternate: 11/22/63: A Novel
11/22/63
Alternate: 11/22/63: A Novel
11/22/63
Alternate: 11/22/63: A Novel
1105 YAKIMA STREET
Alternate: 1
1105 YAKIMA STREET
Alternate: 1
113 MINUTES
Alternate: 113 Minutes (BookShots)
11TH HOUR
Alternate: 11th Hour (Women's Murder Club, 11)
1225 CHRISTMAS TREE LANE
10TH ANNIVERSARY
Alternate: 10th Anniversary (Women's Murder Club, 10)
'TIL DEATH DO US PART, Amanda Quick
10TH ANNIVERSARY
Alternate: 10th Anniversary (Women's Murder Club, 10)
11/22/63
Alternate: 11/22/63: A Novel
11/22/63
Alternate: 11/22/63: A Novel
11/22/63
Alternate: 11/22/63: A Novel
1105 YAKIMA STREET
Alternate: 1
1105 YAKIMA STREET
Alternate: 1
113 MINUTES
Alternate: 113 Minutes (BookShots)
11TH HOUR
Alternate: 11th Hour (Women's Murder Club, 11)
1225 CHRISTMAS TREE LANE
12TH OF NEVER
Alternate: 12th of Never (Women's Murder Cl